# Installing References

In [1]:
!pip install -qU langchain langchain-community langchain-huggingface sentence_transformers faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00


# Loading References

In [ ]:
import getpass
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

Loading CSV file with questions and answer training samples.

In [4]:
loader = CSVLoader(
    "intern_screening_dataset_train.csv"
)
data = loader.load()



# Loading Embedding Model


In [2]:
inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")

Enter your HF Inference API Key:

··········


In [ ]:
def load_embedding_model(model_id: str) -> HuggingFacePipeline:
  return HuggingFaceEmbeddings(model_name=model_id, model_kwargs={'device': 'cuda'})

As an embedding model, it was used the model "sentence-transformers/all-mpnet-base-v2”, a Siamese BERT-Network fined tuned with a large dataset of 1B sentence pairs.

In [6]:
embeddings = load_embedding_model("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformersTokenTextSplitter uses the tokenizer of same model of the embedding: "sentence-transformers/all-mpnet-base-v2”

In [ ]:
text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [8]:
vector_store = FAISS.from_documents(docs, embeddings)
print(vector_store.index.ntotal)

20424


In [9]:
vector_store.save_local("vector_store_index")